# Text Analytics Coursework -- Tasks 1.1 to 1.3

# Instructions:

This notebook is forms the first part of your coursework assignment for Text Analytics in Spring 2025. You will need to read the instructions below and complete numbered tasks indicated by "TASK n". To complete the tasks, you will write code or explanations between the comments "#WRITE YOUR ANSWER HERE" and "#END OF ANSWER". For example:

TASK 0: Complete the function below to output "hello world". 


In [12]:
def demo_fun():
    # WRITE YOUR ANSWER HERE
    print("hello world")
    # END OF ANSWER

### DO NOT MODIFY
demo_fun()
###

hello world


There is also some code in the cell that should not be modified. This code saves your results to file in the correct format, which is necessary for us to be able to mark your answers. Before you submit your notebook, please make sure this code has not been modified, then restart your kernel, clear all cell outputs, run all of your code once again, then save the notebook. 

Please note:
  * The notebook you upload must include all the saved cell output after running all cells.
  * The notebook code must be complete so that it reproduces all your output when we run it. 
  * For this coursework, we recommend that you use your virtual environment that you created for the labs. The packages you need are: numpy, scipy, nltk, pytorch, transformers and datasets (from HuggingFace), pandas, matplotlib and scikit-learn. 

## Marking guidelines:
1. This notebook is worth 32% of the marks for the Text Analytics assignment.
1. The number of marks for each task is shown alongside the task.
1. We will evaluate the output of your code after running it, and marks will be awarded based on how well the output matches the task's instructions. 
1. We will give partial marks for incomplete or partially correct answers. 
1. We do not give additional marks for code style or comments, but clear code will help us to understand what you have done so that we can award partial marks where necessary. 
1. Unless the task asks you to implement something from scratch, there is no penalty for using software libraries in your implementation.

## Support:

The main source of support will be during the lab sessions. The TAs and lecturers will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the assessed tasks, they can only answer clarifying questions about what you have to do. Please email Edwin if you have any other queries edwin.simpson@bristol.ac.uk and/or post your query to the Teams channel for this unit.

## Deadline:

The notebook must be submitted along with the second notebook on Blackboard before **Monday 28th April at 13.00**. 

## Submission:

For this part of the assignment, please zip up the folder containing this file and the 'outputs' directory, containing the output from this notebook as .csv files. Please name your notebook file like this:
   * Name this notebook 'text_analytics_part1_\<student number\>.ipynb'. Replace '\<student number\>' with your student number, which consists only of digits beginning with '2'. 
   * We mark anonymously, so please don't include your name in the notebook.

You can submit the file on Blackboard to the submission point "Text Analytics Part 1 Notebook". Remember that the assignment also has parts 2 and 3, described in the PDF file on Blackboard.

# Setup: random seeds

Each student will work with slightly different data splits and model weights, which will be determined by setting your 'random seed'. 
We will check that your results come from using your random seed. Please set the seed in the cell below by changing the value of 'my_student_number' to your own student number (not your username, the number you can see on eVision that contains only digits). 

Using the correct seed ensures that your results are reproducible when we rerun your notebook.

In [13]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python's built-in random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable benchmark mode for reproducibility

### SET YOUR SEED TO YOUR STUDENT NUMBER HERE
my_student_number = 2532578
set_seed(my_student_number)

# Setup: loading the data

Let's make a folder to save the output of your work:

In [14]:
import os
import pandas as pd

# os.mkdir('./outputs')

Now, let's load some more packages we will need later:

In [15]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the Emotion dataset
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split 


The dataset classifies paragraphs taken from corporate disclosures that discuss climate-related issues. It classifiers them into "risk" (0), "neutral" (1) or "opportunity" (2) representing the sentiment of the paragraph.

First we need to load the data. The data is already split into train, validation and test. The _validation_ set (also called 'development' set or 'devset') can be used to compute performance of your model when tuning hyperparameters, optimising combinations of features, or looking at the errors your model makes before improving it. This allows you to hold out the test set (i.e., not to look at it at all when developing your method) to give a fair evaluation of the model and how well it generalises to new examples. This avoids tuning the model to specific examples in the test set. An alternative approach to validation is to not use a single fixed validation set, but instead use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

In [16]:

cache_dir = "./data_cache"

# load the original training set from HuggingFace
train_dataset = load_dataset(
    "climatebert/   ",
    split="train",
    cache_dir=cache_dir,
)

# we're going to create a new validation set by splitting the data
dataset_splits = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_splits["train"]
val_dataset = dataset_splits["test"]

train_texts = np.array(train_dataset["text"])
val_texts = np.array(val_dataset["text"])

train_labels = np.array(train_dataset["label"])
val_labels = np.array(val_dataset["label"])

print(f"Training dataset with {len(train_texts)} instances loaded")
print(f"Development/validation dataset with {len(val_texts)} instances loaded")

### DO NOT MODIFY
# save gold labels to file
pd.DataFrame(val_labels).to_csv('./outputs/val_labels.csv')

Training dataset with 800 instances loaded
Development/validation dataset with 200 instances loaded


In this notebook, you're going to build three different classifiers for this dataset, then compare how they work, and analyse the results. We are going to start by implementing a naïve Bayes classifier from scratch. 

We are going to begin by initialising some useful variables and doing some very simple pre-processing using CountVectorizer.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

num_classes = 3

def preprocess(train_texts):
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess(train_texts)

d:\Anaconda\envs\text_analytics\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## TASK 1.1a

Complete the function below to compute the class priors, $p(y_n = c)$ for each class label $c$, where $y_n$ is the class label of document $n$. Do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the class priors as a list or Numpy array containing the probabilities. You do not need to apply any smoothing or regularisation.    (3 marks)

In [18]:
def compute_class_priors(texts, labels):
    priors = np.zeros(num_classes)

    ### WRITE YOUR ANSWER HERE
    
    # Count occurrences of each class
    for c in range(num_classes):
        priors[c] = np.sum(labels == c)
    
    # Normalize to get probabilities
    priors = priors / len(labels)
    
    ### END OF ANSWER
    return priors


class_priors = compute_class_priors(train_texts, train_labels)
print(class_priors)

### DO NOT MODIFY
pd.DataFrame(class_priors).to_csv('./outputs/11a_class_priors.csv')

[0.33625 0.4125  0.25125]


## TASK 1.1b

Complete the function below to extract unigram and bigram features from the text, then compute the liklihood $p(x_{ni} = w | y_n = c)$ that the $i$ th n-gram in document $n$ is $w$, given that the class of $n$ is $c$. Again, do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the likelihoods as a 2D Numpy array containing probabilities. You should apply smoothing by adding counts of +1 to the counts of each feature.  (3 marks)

In [19]:
def compute_feature_likelihoods(X, labels):

    likelihoods = np.ones((num_features, num_classes))  # a 2D numpy array where you can store the likelihoods. Note that all values are initialised to one.

    ### WRITE YOUR ANSWER HERE
    # Count feature occurrences per class
    for c in range(num_classes):
        # Get documents of class c
        X_c = X[labels == c]
        
        # Sum counts of each feature in class c (add 1 for smoothing)
        feature_counts = np.sum(X_c, axis=0) + 1  # Laplace smoothing
        
        # Total count of features in class c (including smoothing)
        total_counts = np.sum(feature_counts)
        
        # Compute likelihoods
        likelihoods[:, c] = feature_counts / total_counts
    

    ### END OF ANSWER
    return likelihoods

likelihoods = compute_feature_likelihoods(X, train_labels)

### DO NOT MODIFY
pd.DataFrame(likelihoods).to_csv('./outputs/11b_likelihoods.csv')

Now, we are going to use the code in the next cell to compute the log probabilities of each class for each text in the validation set. This code will use the previous functions you implemented, compute_class_priors and compute_feature_likelihoods. The log probabilities will be stored in the 'predictions' array.

In [20]:
from scipy.special import logsumexp


def NB_classify(class_priors, likelihoods, X_val):

    predictions = np.zeros((X_val.shape[0], num_classes))  # an empty numpy array to store the predictions in

    sum_of_log_likelihoods = X_val.dot(np.log(likelihoods))
    log_joint_prob = sum_of_log_likelihoods + np.log(class_priors)[None, :]
    for n, doc in enumerate(X_val):
        predictions[n, :] = log_joint_prob[n]
        predictions[n, :] -= logsumexp(predictions[n, :])
    return predictions

predictions = NB_classify(class_priors, likelihoods, X_val)
print(predictions)

[[-7.80655858e+00 -3.06319054e-03 -5.93264965e+00]
 [ 0.00000000e+00 -4.88670735e+01 -6.57710493e+01]
 [-8.81538067e+00 -7.46923231e-02 -2.63355620e+00]
 [ 0.00000000e+00 -4.47798040e+01 -5.81591208e+01]
 [-1.66662274e+01 -6.39205671e-05 -9.65880608e+00]
 [-1.45317649e+00 -3.43727550e+00 -3.09216992e-01]
 [-1.07239238e+00 -4.18837477e-01 -1.82059036e+01]
 [ 0.00000000e+00 -3.81072853e+01 -5.01124430e+01]
 [-3.87037293e+00 -9.21703841e-01 -5.42473163e-01]
 [-1.43222678e-09 -2.06170969e+01 -2.18619789e+01]
 [-1.80934958e-02 -4.16844179e+00 -6.00985888e+00]
 [-1.99984535e+01 -2.06432560e-09 -3.71154677e+01]
 [-8.16569364e+00 -2.84279825e-04 -3.24565515e+01]
 [-2.93506582e+01 -1.69001136e+01 -4.57483793e-08]
 [-4.99875341e-09 -2.01010750e+01 -1.95803981e+01]
 [-2.26145179e+01 -1.27937483e-09 -2.06024001e+01]
 [-1.62777696e+01 -1.16272994e+00 -3.74884934e-01]
 [-1.22335808e-02 -4.43164596e+00 -8.23895655e+00]
 [-1.96239659e+01 -1.31095805e+01 -2.02873389e-06]
 [-7.32345676e+00 -2.83394139e+

Use the 'predictions' array above to compute and print the accuracy of the classifier on the validation set.   

In [21]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, np.argmax(predictions, axis=1))

0.735

## TASK 1.1c

The simplicty of naïve Bayes means that we can quite easily interpret the model. In the code above, we used the functions you implemented, compute_feature_likelihoods and compute_class_priors, to train an NB classifier with our training set. Given this classifier, which are the five n-gram features that most strongly indicate that the document belongs to class 0? Store these features in the 'top_features' list below.    (4 marks)

In [22]:
top_features = []

### WRITE YOUR ANSWER HERE

# Get feature name list
feature_names = vectorizer.get_feature_names_out()

# Get the likelihoods for class 0
class_0_likelihoods = likelihoods[:, 0]

# Get the top 5 feature indices (highest likelihoods)
top_feature_indices = np.argsort(class_0_likelihoods)[::-1][:5]

# Map indices to actual n-grams
top_features = [feature_names[i] for i in top_feature_indices]
### END OF ANSWER

### DO NOT MODIFY
print(top_features)
pd.DataFrame(top_features).to_csv('./outputs/11c_top_feats.csv')
###

['climate change', 'of the', ', and', 'in the', 'to the']


Up to this point, the classifier used bigrams features extracted using CountVectorizer with the wordnet tokenizer. 

## TASK 1.1d

Your task is to improve the naïve Bayes classifier by changing the preprocessing or features only. It is up to you to decide how many changes are needed to improve the classifier -- a single change may be enough to achieve a good result (and maximum marks) and you should only include steps that help performance. Complete the 'preprocess_improved' function below, and complete the code at the end of the cell to compute accuracy of the improved classifier on the validation set.     (3 marks)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE
    
    def custom_tokenizer(text):
        # use lower case text only
        tokens = word_tokenize(text.lower())
        return [t for t in tokens 
                # filter English stopwords
                if t not in stopwords.words('english')
                # filter punctuations 
                and t not in string.punctuation]

    # Improved vectorizer with TF-IDF weighting
    vectorizer = TfidfVectorizer(
        # enlarge ngam range, include Unigrams + bigrams
        ngram_range=(1, 10),
        tokenizer=custom_tokenizer,
        # Max features to reduce noise
        max_features=10000
    )
    
    X = vectorizer.fit_transform(train_texts)
    num_features = X.shape[1]
    
    # Process validation set with same vectorizer
    X_val = vectorizer.transform(val_texts)
    
    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

d:\Anaconda\envs\text_analytics\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.785


## TASK 1.2

Below is an implementation of a neural network classifier that we can apply to the same dataset. However, there are some mistakes in the code and some poor choices in the choice of parameters and architecture. Your task is to fix the errors, make better parameter choices, and improve the model's performance. **Modify the code within the next cell** to improve the neural network classifier, then run it and compute its accuracy using the code in the cell after that.   (8 marks)

original: 0.365

In [24]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE: MODIFY THE CODE WITHIN THIS CELL
 
from torch import nn
from torch import optim
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")  

sequence_length = 256  # Increased to capture more context
batch_size = 64  # Increased for stable training

def tokenize_function(examples):
    return tokenizer(
        examples["text"],  # Adjust the key based on your dataset structure
        padding="max_length",  # Ensures equal sequence lengths
        truncation=True,       # Truncates longer sequences
        max_length=sequence_length,        # Adjust as needed
        return_tensors="pt"
    )

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"]) # Adjust column names
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"]) # Adjust column names
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False)

class FFTextClassifier(nn.Module):
    
    def __init__(self, vocab_size, sequence_length, num_classes, 
                 embedding_size=384, hidden_size=512, dropout=0.4):  # Enlarge model capacity with larger dropout
        super(FFTextClassifier, self).__init__()

        # Embedding layer with dropout
        self.embedding = nn.Embedding(vocab_size, embedding_size, padding_idx=tokenizer.pad_token_id)
        self.embed_dropout = nn.Dropout(dropout)
        
        # Add LSTM
        self.lstm = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            num_layers=3,            # set 3 layers
            bidirectional=True,     # use bidirectional
            batch_first=True,
            dropout=dropout if 3 > 1 else 0  # Inter-layer dropout
        )
        
        # Enhanced classification head
        self.fc = nn.Sequential(
            nn.Linear(hidden_size*2, hidden_size),  # Concatenate bidirectional outputs
            nn.BatchNorm1d(hidden_size),            # Add BatchNorm layer
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, input_words):
        # Remove previous embedding method and use dropout version to reduce overfitting
        emb = self.embed_dropout(self.embedding(input_words))
        
        lstm_out, _ = self.lstm(emb)
        
        # Use mean pooling instead of last hidden state
        pooled = lstm_out.mean(dim=1) 
        
        output = self.fc(pooled)

        # Notice we haven't applied a softmax activation to the output layer -- it's not required by Pytorch's loss function.
        
        return output

    def run_training(self, train_dataloader, dev_dataloader):
        
        # training hyperparameters
        num_epochs = 15              # Increase number of epochs
        learning_rate = 3e-4         # Modify learning rate
        weight_decay = 1e-4          # Add L2 regularization
        grad_clip = 1.0              # Add gradient clipping threshold

        loss_fn = nn.CrossEntropyLoss() # create loss function object

        # Switch to AdamW method and add weight_decay
        optimizer = optim.AdamW(self.parameters(), lr=learning_rate, weight_decay=weight_decay) # create the optimizer
        
        # Applies cosine annealing learning rate scheduling 
        # gradually adjusts LR following a cosine curve over epochs to help escape local minima and improve convergence
        scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)  

        # Original version only display the losses
        dev_losses = []
        # Here I want to check accuracy in each epoches and record the best model
        best_acc = 0
        # Adjust the early stop parameters
        early_stop_patience = 5
        no_improve = 0

        for e in range(num_epochs):
            # Track performance on the training set as we are learning...
            train_losses = [] 

            self.train()  # Put the model in training mode.
            
            for batch in train_dataloader:
                # Iterate over each batch of data

                optimizer.zero_grad() # Reset the optimizer

                # Use the model to perform forward inference on the input data.
                output = self(batch['input_ids'])

                # Compute the loss for the current batch of data
                batch_loss = loss_fn(output, batch['label'])
                
                # Perform back propagation to compute the gradients with respect to each weight
                batch_loss.backward()

                # Gradient clipping
                nn.utils.clip_grad_norm_(self.parameters(), grad_clip)

                # Update the weights using the compute gradients
                optimizer.step()
                
                # Record the loss from this sample to keep track of progress.
                train_losses.append(batch_loss.item())
            
            print("Epoch: {}/{}".format((e+1), num_epochs),
                "Training Loss: {:.4f}".format(np.mean(train_losses)))
            
            self.eval() # Switch model to evaluation mode

            dev_losses_epoch = []

            val_loss, correct = 0, 0

            with torch.no_grad():
                for batch in dev_dataloader:
                    output = self(batch['input_ids'])

                    dev_loss = loss_fn(output, batch['label'])

                    val_loss += loss_fn(output, batch['label']).item()
                    pred = output.argmax(dim=1)
                    correct += (pred == batch['label']).sum().item()

                    # Save the loss on the dev set
                    dev_losses_epoch.append(dev_loss.item())
            
            acc = correct / len(dev_dataloader.dataset)
            scheduler.step()
            
            # Early stopping mechanism
            if acc > best_acc:
                best_acc = acc
                no_improve = 0
                torch.save(self.state_dict(), 'best_model.pt')
            else:
                no_improve += 1
                if no_improve >= early_stop_patience:
                    print(f"Early stopping at epoch {e+1}")
                    break
            
            dev_losses.append(np.mean(dev_losses_epoch))
                    
            print("Epoch: {}/{}".format((e+1), num_epochs),
                "Validation Loss: {:.4f}".format(dev_losses[-1]) )
            
            print(f"Epoch {e+1}: Val Acc {acc:.4f}")

        # Load best model
        self.load_state_dict(torch.load('best_model.pt', weights_only=True))
        return best_acc

def predict_nn(trained_model, data_loader):

    trained_model.eval()

    pred_labs = [] # predicted labels to return

    with torch.no_grad():
        for batch in data_loader:
            output = trained_model(batch['input_ids'])
            pred_labs.extend(output.argmax(1).tolist())

    return pred_labs

vocab_size = len(tokenizer)
nn_classifier_model = FFTextClassifier(vocab_size, sequence_length, num_classes)
best_acc = nn_classifier_model.run_training(train_loader, val_loader)

predictions_nn = predict_nn(nn_classifier_model, val_loader)

### END OF ANSWER 

### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)

Epoch: 1/15 Training Loss: 0.9753
Epoch: 1/15 Validation Loss: 1.0934
Epoch 1: Val Acc 0.3750
Epoch: 2/15 Training Loss: 0.6567
Epoch: 2/15 Validation Loss: 1.0589
Epoch 2: Val Acc 0.6150
Epoch: 3/15 Training Loss: 0.5111
Epoch: 3/15 Validation Loss: 0.9973
Epoch 3: Val Acc 0.6750
Epoch: 4/15 Training Loss: 0.4207
Epoch: 4/15 Validation Loss: 0.8722
Epoch 4: Val Acc 0.7550
Epoch: 5/15 Training Loss: 0.3719
Epoch: 5/15 Validation Loss: 0.7188
Epoch 5: Val Acc 0.7700
Epoch: 6/15 Training Loss: 0.3324
Epoch: 6/15 Validation Loss: 0.5881
Epoch 6: Val Acc 0.7250
Epoch: 7/15 Training Loss: 0.2736
Epoch: 7/15 Validation Loss: 0.6099
Epoch 7: Val Acc 0.7050
Epoch: 8/15 Training Loss: 0.2639
Epoch: 8/15 Validation Loss: 0.7069
Epoch 8: Val Acc 0.7350
Epoch: 9/15 Training Loss: 0.2019
Epoch: 9/15 Validation Loss: 0.5871
Epoch 9: Val Acc 0.7350
Epoch: 10/15 Training Loss: 0.1688
Early stopping at epoch 10
0.77


We now explore the use of transformers for building a text classifier. First, let's look at how the process a document. We'll chose one at random from the training set:

In [25]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]

## TASK 1.3a

Use the HuggingFace transformers library to load the pretrained BERT model "prajjwal1/bert-tiny". Obtain a document embedding for the chosen document given above. Comment your code to explain how it obtains a representation of the document.    (3 marks)

In [26]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoTokenizer, AutoModel
import torch

doc_emb = []  # use this variable to store the document embedding

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

# Tokenize the chosen document
inputs = tokenizer(
    chosen_document,
    return_tensors="pt",       # Return PyTorch tensors
    truncation=True,           # Truncate long inputs to model max length
    padding=True,              # Pad shorter inputs for consistent shape
    max_length=512             # BERT's maximum sequence length
)

# Forward pass to get hidden states
with torch.no_grad():  # No need to compute gradients for inference
    outputs = model(**inputs)

# Get the last hidden states (token embeddings)
last_hidden_state = outputs.last_hidden_state

# Mean pooling over the sequence length dimension to get a fixed-size document embedding
# This ignores padding tokens
doc_embedding = last_hidden_state.mean(dim=1)

# Convert to a list for saving
doc_emb = doc_embedding.squeeze().tolist()

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(doc_emb).to_csv('./outputs/13a_sen_emb.csv')
print(doc_emb)
###

[-0.37442052364349365, 1.143388271331787, -0.5592844486236572, -0.5962960124015808, 0.3182202875614166, 0.017382092773914337, -0.7227636575698853, 0.38975292444229126, -1.882852554321289, -0.2673739492893219, 0.5706955194473267, 0.47942984104156494, -0.7306100130081177, -0.622067391872406, 1.216062068939209, -0.7176520824432373, -1.1777868270874023, 0.11069127172231674, -0.8704712986946106, 0.2983161211013794, 0.002907872200012207, -0.20897214114665985, 0.7724873423576355, 1.6112291812896729, 2.2291364669799805, -0.28930729627609253, 0.27317458391189575, 0.14854484796524048, -0.17756770551204681, -0.45032772421836853, -0.9861768484115601, -2.2329983711242676, -0.40728023648262024, -0.06895917654037476, 0.23049230873584747, -0.27631446719169617, -0.5073726773262024, 0.8247656226158142, -1.6947153806686401, -0.1751004010438919, 0.8689639568328857, -0.2957328259944916, -0.08509688079357147, -1.2897292375564575, 0.5441653728485107, -1.404317021369934, -0.5225303173065186, 0.714616119861602

## TASK 1.3b

Using the same document embeddings method as the previous task (1.3a), find the most similar document to the 'chosen_document' from within the validation set (from the 'val_texts' object). Use a standard similarity metric that considers the direction but not the magnitude of the embedding vectors.  (2 marks)

In [27]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

most_similar_doc = ""  # use this variable to store the most similar document

tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")
model.eval()

# Generate document embedding using mean pooling strategy from 1.3a
def doc_embedding_13a(text):
    inputs = tokenizer(
        chosen_document,
        return_tensors="pt",       # Return PyTorch tensors
        truncation=True,           # Truncate long inputs to model max length
        padding=True,              # Pad shorter inputs for consistent shape
        max_length=512             # BERT's maximum sequence length
    )
    with torch.no_grad():
        outputs = model(**inputs)
    hidden_states = outputs.last_hidden_state  # shape: (1, seq_len, hidden_size)
    embedding = hidden_states.mean(dim=1)      # mean pooling
    return embedding.squeeze()                 # shape: (hidden_size,)

# Get target document embedding
chosen_embedding = doc_embedding_13a(chosen_document)

# Set cosine similarities variables
best_score = -1.0
most_similar_doc = ""

# Generate embeddings for all validation documents
for text in val_texts:
    val_emb = doc_embedding_13a(text)
    
    # Calculate cosine similarities (direction-based metric)
    similarity = F.cosine_similarity(chosen_embedding, val_emb, dim=0).item()

    if similarity > best_score:
        best_score = similarity
        most_similar_doc = text

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame([chosen_document, most_similar_doc]).to_csv("./outputs/13b_most_similar.csv")
print(chosen_document)
print(most_similar_doc)
###

The Group is not aware of any noise pollution that could negatively impact the environment, nor is it aware of any impact on biodiversity. With regards to land use, the Group is only a commercial user, and the Group is not aware of any local constraints with regards to water supply. The Group does not believe that it is at risk with regards to climate change in the near-or mid-term.
For an airline, fuel right-sizing can largely be seen as a capital allocation strategy. We have a consistent strategy of investing in fuel-saving new airframes and engines, which are the largest factors in fuel consumption. In 2018, we made two major announcements related to fuel burn: A220s and New Engine Options. More information on these can be found on page 23.


## TASK 1.3c

Implement a classifier based on the same pretrained transformer model, "prajjwal1/bert-tiny". Evaluate your model's performance on the validation set. Use an 'auto class' from HuggingFace to build your classifier (see https://huggingface.co/docs/transformers/model_doc/auto).   (6 marks)

In [28]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE

predictions_bert = []  # use this variable to store the predicted labels for the validation set

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score

# Load pretrained model and tokenizer with Auto Classes
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=num_classes)

# Tokenization function for dataset preprocessing
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

# Prepare datasets and tokenization
train_ds = Dataset.from_dict({"text": train_texts, "label": train_labels}).map(tokenize_function, batched=True)
val_ds = Dataset.from_dict({"text": val_texts, "label": val_labels}).map(tokenize_function, batched=True)

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Configure training parameters
training_args = TrainingArguments(
    output_dir="./result-of-bert-tiny",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=15,
    evaluation_strategy="epoch",
    logging_steps=20,
    seed=my_student_number
)

# Metric computation for validation
def compute_acc(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_acc
)

# Fine-tune the model
trainer.train()

# Generate predictions
predictions = trainer.predict(val_ds)
predictions_bert = np.argmax(predictions.predictions, axis=1).tolist()

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_tinybert = accuracy_score(val_dataset["label"], predictions_bert)
print(accuracy_tinybert)
###

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

d:\Anaconda\envs\text_analytics\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.069400,1.034554,0.390000
2,1.039500,0.987520,0.445000
3,0.995100,0.916494,0.720000
4,0.883000,0.842782,0.740000
5,0.818200,0.794942,0.745000
6,0.798300,0.751429,0.780000
7,0.750000,0.716806,0.795000
8,0.678600,0.688723,0.790000
9,0.659800,0.663436,0.790000
10,0.648500,0.643694,0.800000


0.8
